# Natural Language Processing Exercise 1

In the first exercise, you will tokenize text in a dataset. This tokenized text will later be used in predictive models. We'll also do preprocessing steps such as stemming and lemmatizing. I should note that this always isn't the best. Also removing stop words, or some stop words. The end result should be data that is ready to be fed into a sklearn pipeline and model.


Potential dataset on sms spam https://www.kaggle.com/uciml/sms-spam-collection-dataset

This is a good use case because people are getting more and more robocalls and spam texts. So say you work at a telecom company and trying to filter out spam texts getting to your customers.

In [2]:
import nltk
import pandas as pd

In [9]:
data = pd.read_csv('../data/spam.csv', encoding = "ISO-8859-1")
data = data.iloc[:, :2]

In [10]:
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [26]:
corpus = data['v2']

In [24]:
from string import punctuation
def tokenize(string):
    # lowercase
    lowered = string.lower()
    
    # remove punctuation
    punc_trans = str.maketrans(punctuation, ' '*len(punctuation))
    cleaned = lowered.translate(punc_trans)
    
    # now split into tokens
    tokens = cleaned.split()
    
    return tokens

In [36]:
def tokenize(string):
    # lowercase
    lowered = string.lower()
    
    tokens = nltk.tokenize.wordpunct_tokenize(lowered)
    return tokens

In [37]:
tokenized = [tokenize(message) for message in corpus]

In [38]:
tokenized[0]

['go',
 'until',
 'jurong',
 'point',
 ',',
 'crazy',
 '..',
 'available',
 'only',
 'in',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 '...',
 'cine',
 'there',
 'got',
 'amore',
 'wat',
 '...']

In [40]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

# TODO: We'll need to figure out a way to include the wordnet corpus with the kernel
# without downloading it in the kernel itself
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/mat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [41]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

In [42]:
def lemmatize_stem(word):
    word = lemmatizer.lemmatize(word, pos='v')
    word = stemmer.stem(word)
    return word

processed = [[lemmatize_stem(word) for word in message] for message in tokenized]

In [43]:
processed

[['go',
  'until',
  'jurong',
  'point',
  ',',
  'crazi',
  '..',
  'avail',
  'onli',
  'in',
  'bugi',
  'n',
  'great',
  'world',
  'la',
  'e',
  'buffet',
  '...',
  'cine',
  'there',
  'get',
  'amor',
  'wat',
  '...'],
 ['ok', 'lar', '...', 'joke', 'wif', 'u', 'oni', '...'],
 ['free',
  'entri',
  'in',
  '2',
  'a',
  'wkli',
  'comp',
  'to',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  '21st',
  'may',
  '2005',
  '.',
  'text',
  'fa',
  'to',
  '87121',
  'to',
  'receiv',
  'entri',
  'question',
  '(',
  'std',
  'txt',
  'rate',
  ')',
  't',
  '&',
  'c',
  "'",
  's',
  'appli',
  '08452810075over18',
  "'",
  's'],
 ['u',
  'dun',
  'say',
  'so',
  'earli',
  'hor',
  '...',
  'u',
  'c',
  'alreadi',
  'then',
  'say',
  '...'],
 ['nah',
  'i',
  'don',
  "'",
  't',
  'think',
  'he',
  'go',
  'to',
  'usf',
  ',',
  'he',
  'live',
  'around',
  'here',
  'though'],
 ['freemsg',
  'hey',
  'there',
  'darl',
  'it',
  "'",
  's',
  'be',
  '3',
  'week',